In [1]:
# Import necessary libraries
import os
import MDAnalysis as mda
import nglview as nv
from IPython.display import display
import glob
import numpy as np
from MDAnalysis.lib.distances import distance_array

def load_trajectory(gro, xtc):
    """
    Load the trajectory using MDAnalysis.
    """
    return mda.Universe(gro, xtc)

def select_atoms(universe):
    """
    Select atoms excluding solvent and ions.
    Modify the selection string based on your system.
    """
    # Exclude water (SOL) and ions (NA, CL)
    selection = universe.select_atoms('not resname SOL') #'not resname SOL NA CL'
    return selection

def resolve_path(pattern):
    """
    Resolve a file path pattern with wildcards to the actual file path.
    
    Parameters:
        pattern (str): The file path pattern containing wildcards.
        
    Returns:
        str: The resolved file path.
        
    Raises:
        FileNotFoundError: If no files match the pattern.
        ValueError: If multiple files match the pattern.
    """
    matched_files = glob.glob(pattern)
    if not matched_files:
        raise FileNotFoundError(f"No files match the pattern: {pattern}")
    elif len(matched_files) > 1:
        raise ValueError(f"Multiple files match the pattern: {pattern}\nMatched files: {matched_files}")
    return matched_files[0]

def visualize_simulation(sim_id):
    """
    Visualize the selected simulation using nglview.
    """
    # Retrieve file paths from the dictionary
    paths = simulation_paths.get(sim_id)
    if not paths:
        print(f"Simulation ID '{sim_id}' not found.")
        return
    
    gro_file = resolve_path(paths['gro_file'])
    xtc_file = paths['xtc_file']
    
    # Load the trajectory
    universe = load_trajectory(gro_file, xtc_file)
    
    # Select atoms (excluding solvent and ions)
    selection = select_atoms(universe)
    
    print(f"Number of atoms selected: {len(selection)}")
    
    # Create an nglview widget with the Universe object
    view = nv.show_mdanalysis(selection)
    
    # 3. Ball+Stick representation for specific elements (excluding solvent and ions)
    view.add_representation('ball+stick', selection='not resname SOL NA CL')
    
    # Center the view on the molecule
    view.center()
    
    # Display the widget
    display(view)

In [ ]:
process_dir = "/home/johannal96/Publications.nobackup/2025/electrofit-ip6-paper-2025/ip6-mieser/process"

simulation_paths = {
    "IP_101101_50": {
        "gro_file": f"{process_dir}/IP_101101_50/run_gmx_simulation/md.gro",
        "xtc_file": f"{process_dir}/IP_101101_50/run_gmx_simulation/md_center.xtc",
    },
        "IP_101101_100": {
        "gro_file": f"{process_dir}/IP_101101_100/run_gmx_simulation/md.gro",
        "xtc_file": f"{process_dir}/IP_101101_100/run_gmx_simulation/md_center.xtc",
    },
}

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact

# Create a dropdown widget for simulation selection
simulation_selector = widgets.Dropdown(
    options=list(simulation_paths.keys()),
    value=list(simulation_paths.keys())[0],  # Default value
    description='Simulation:',
    disabled=False,
)


# Use interact to link the dropdown to the visualization function
interact(visualize_simulation, sim_id=simulation_selector);

interactive(children=(Dropdown(description='Simulation:', options=('IP_101101_50', 'IP_101101_100'), value='IP…